In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from astropy.table import Table, vstack
from astroquery.vizier import Vizier
from astropy import units as u

import swasputils

In [3]:
CATALOGUES = [
    'J/BaltA/9/646/catalog',
    'J/ApJS/247/44/table2',
]

In [4]:
source_table = Table.read(os.path.join(swasputils.CACHE_LOCATION, 'source_coords.fits'))
source_table = source_table[:1000]

In [5]:
results = []
SPLIT_SIZE = 1000
for i in range(int(len(source_table) / SPLIT_SIZE) + 1):
    sources = source_table[i * SPLIT_SIZE : (i+1) * SPLIT_SIZE]
    if len(sources) > 0:
        results.append(Vizier.query_region(sources, radius=2*u.arcmin, catalog=CATALOGUES))

In [6]:
stacked_results = {}
for catalogue in CATALOGUES:
    stacked_results[catalogue] = []
    for result in results:
        try:
            stacked_results[catalogue].append(result[catalogue])
        except TypeError:
            pass
    try:
        stacked_results[catalogue] = vstack(stacked_results[catalogue])
    except ValueError:
        pass

In [7]:
stacked_results['J/ApJS/247/44/table2']

_q,LC,_2MASS,f_2MASS,RAJ2000,DEJ2000,Rmag,Jmag,Hmag,Ksmag,Per,E_Per,e_Per,SNLS,FileName,Simbad
,,,,"""h:m:s""","""d:m:s""",mag,mag,mag,mag,d,d,d,,,
int32,str2,str17,uint8,str11,str12,float32,float32,float32,float32,int16,int16,int16,float32,str67,str6
1492,LC,J00020390+5358588,0,00 02 03.91,+53 58 58.88,11.05,6.22,5.22,4.83,125,132,119,2.08,KELT_N16_lc_044840_V01_east_raw_KELT_N16_lc_039841_V01_west_raw.dat,Simbad


In [8]:
stacked_results['J/BaltA/9/646/catalog']

[]